## Importing Datasets

In [131]:
import pandas as pd

orders_df = pd.read_csv("List of Orders.csv", parse_dates=["Order Date"])
order_details_df = pd.read_csv("Order Details.csv")
sales_target_df = pd.read_csv("Sales target.csv")

In [132]:
orders_df.columns = orders_df.columns.str.strip().str.replace(' ', '')
order_details_df.columns = order_details_df.columns.str.strip().str.replace(' ', '').str.replace('-','')
sales_target_df.columns = sales_target_df.columns.str.strip().str.replace(' ', '')

In [133]:
sales_target_df["MonthofOrderDate"] = pd.to_datetime(
    sales_target_df["MonthofOrderDate"], format="%b-%y"
)

In [134]:
sales_target_df["MonthofOrderDate"] = sales_target_df["MonthofOrderDate"].dt.strftime("%Y-%m-%d")

In [135]:
orders_df["OrderDate"] = pd.to_datetime(orders_df["OrderDate"], errors="coerce", dayfirst=True)
orders_df["OrderDate"] = orders_df["OrderDate"].dt.strftime("%Y-%m-%d")

In [136]:
sales_target_df.head()


,MonthofOrderDate,Category,Target
0,2018-04-01,Furniture,10400.0
1,2018-05-01,Furniture,10500.0
2,2018-06-01,Furniture,10600.0
3,2018-07-01,Furniture,10800.0
4,2018-08-01,Furniture,10900.0


## SQL Queries

In [155]:
import sqlite3

conn = sqlite3.connect("mydb.sqlite")

In [156]:
conn.execute("DROP TABLE IF EXISTS order_details;")
conn.execute("DROP TABLE IF EXISTS orders;")
conn.execute("DROP TABLE IF EXISTS sales_target;")

In [157]:
conn.execute("""
CREATE TABLE orders (
    OrderID TEXT PRIMARY KEY,
    OrderDate DATE,
    CustomerName TEXT,
    State TEXT,
    City TEXT
);
""")


In [158]:
conn.execute("""
CREATE TABLE order_details (
    DetailID INTEGER PRIMARY KEY AUTOINCREMENT,
    OrderID TEXT,
    Amount REAL,
    Profit REAL,
    Quantity INT,
    Category TEXT,
    SubCategory TEXT,
    FOREIGN KEY (OrderID) REFERENCES orders(OrderID)
);
""")


In [159]:
conn.execute("""
CREATE TABLE sales_target (
    MonthofOrderDate DATE,
    Category TEXT,
    Target REAL,
    PRIMARY KEY (MonthofOrderDate, Category)
);
""")

In [160]:
orders_df.to_sql("orders", conn, if_exists="append", index=False)
order_details_df.to_sql("order_details", conn, if_exists="append", index=False)
sales_target_df.to_sql("sales_target", conn, if_exists="append", index=False)

36

### Sale & Revenue Analysis

Total Sales by Category:

In [161]:
query = """
SELECT Category, SUM(Amount * Quantity) AS TotalSales
FROM order_details
GROUP BY Category
ORDER BY TotalSales DESC;
"""
pd.read_sql(query, conn)

,Category,TotalSales
0,Electronics,816583.0
1,Furniture,665765.0
2,Clothing,664522.0


Monthly Sales Trend:

In [162]:
query = """
SELECT strftime('%Y-%m', o.OrderDate) AS Month,
       SUM(od.Amount * od.Quantity) AS Sales
FROM orders o
JOIN order_details od ON o.OrderID = od.OrderID
GROUP BY Month
ORDER BY Month;
"""
pd.read_sql(query, conn)

,Month,Sales
0,2018-04,152827.0
1,2018-05,138767.0
2,2018-06,110015.0
3,2018-07,53144.0
4,2018-08,151514.0
5,2018-09,132717.0
6,2018-10,176663.0
7,2018-11,229785.0
8,2018-12,164839.0
9,2019-01,337229.0


Top 10 Products by Sales

In [163]:
query = """
SELECT od.SubCategory, SUM(od.Amount * od.Quantity) AS TotalSales
FROM order_details od
GROUP BY od.SubCategory
ORDER BY TotalSales DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,SubCategory,TotalSales
0,Printers,307963.0
1,Bookcases,295598.0
2,Saree,263523.0
3,Chairs,206479.0
4,Electronic Games,204850.0
5,Phones,200893.0
6,Trousers,124640.0
7,Accessories,102877.0
8,Tables,90706.0
9,Stole,86155.0


### Profit Analysis

Profit by Category:

In [164]:
query = """
SELECT Category, SUM(Profit) AS TotalProfit
FROM order_details
GROUP BY Category
ORDER BY TotalProfit DESC;
"""
pd.read_sql(query, conn)

,Category,TotalProfit
0,Clothing,11163.0
1,Electronics,10494.0
2,Furniture,2298.0


Products generating Loss

In [165]:
query = """
SELECT SubCategory, SUM(Profit) AS Loss
FROM order_details
GROUP BY SubCategory
HAVING Loss < 0;
"""
pd.read_sql(query, conn)

,SubCategory,Loss
0,Electronic Games,-1236.0
1,Tables,-4011.0


Profit Margin%:

In [166]:
query = """
SELECT Category,
       SUM(Profit) / SUM(Amount * Quantity) * 100 AS ProfitMargin
FROM order_details
GROUP BY Category
ORDER BY ProfitMargin DESC;
"""
pd.read_sql(query, conn)

,Category,ProfitMargin
0,Clothing,1.679854
1,Electronics,1.285111
2,Furniture,0.345167


### Customer Analysis

Top 10 Customers by Sales:

In [167]:
query = """
SELECT o.CustomerName, SUM(od.Amount * od.Quantity) AS TotalSales
FROM orders o
JOIN order_details od ON o.OrderID = od.OrderID
GROUP BY o.CustomerName
ORDER BY TotalSales DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,CustomerName,TotalSales
0,Yaanvi,103435.0
1,Seema,64222.0
2,Soumya,46086.0
3,Pooja,43222.0
4,Shishu,38896.0
5,Abhishek,35521.0
6,Surabhi,35362.0
7,Swapnil,35019.0
8,Sarita,31837.0
9,Mukesh,30708.0


Number of Orders per Customer:

In [168]:
query = """
SELECT o.CustomerName, COUNT(DISTINCT o.OrderID) AS NumOrders
FROM orders o
JOIN order_details od ON o.OrderID = od.OrderID
GROUP BY o.CustomerName
ORDER BY NumOrders DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,CustomerName,NumOrders
0,Shreya,6
1,Shubham,5
2,Pooja,5
3,Abhishek,5
4,Yogesh,4
5,Surabhi,4
6,Shruti,4
7,Sheetal,4
8,Rohan,4
9,Priyanka,4


### Regional Analysis

Sales by State:

In [169]:
query = """
SELECT State, SUM(od.Amount * od.Quantity) AS TotalSales
FROM orders o
JOIN order_details od ON o.OrderID = od.OrderID
GROUP BY State
ORDER BY TotalSales DESC;
"""
pd.read_sql(query, conn)

,State,TotalSales
0,Madhya Pradesh,569685.0
1,Maharashtra,467660.0
2,Uttar Pradesh,150032.0
3,Gujarat,100292.0
4,Delhi,97071.0
5,Rajasthan,94050.0
6,Andhra Pradesh,82897.0
7,Punjab,77591.0
8,Karnataka,66231.0
9,West Bengal,58035.0


Sales by City:

In [170]:
query = """
SELECT City, SUM(od.Amount * od.Quantity) AS TotalSales
FROM orders o
JOIN order_details od ON o.OrderID = od.OrderID
GROUP BY City
ORDER BY TotalSales DESC;
"""
pd.read_sql(query, conn)

,City,TotalSales
0,Indore,455989.0
1,Mumbai,293206.0
2,Pune,174454.0
3,Allahabad,120812.0
4,Chandigarh,115237.0
5,Delhi,106037.0
6,Bhopal,104730.0
7,Hyderabad,82897.0
8,Ahmedabad,69346.0
9,Bangalore,66231.0


### Target vs Actual Analysis

In [171]:
query = """
SELECT strftime('%Y-%m', s.MonthofOrderDate) AS Month,
       s.Category,
       s.Target,
       SUM(od.Amount * od.Quantity) AS ActualSales,
       SUM(od.Amount * od.Quantity) - s.Target AS Difference
FROM sales_target s
LEFT JOIN orders o
  ON strftime('%Y-%m', o.OrderDate) = strftime('%Y-%m', s.MonthofOrderDate)
LEFT JOIN order_details od
  ON o.OrderID = od.OrderID
 AND od.Category = s.Category
GROUP BY s.MonthofOrderDate, s.Category, s.Target
ORDER BY s.MonthofOrderDate, s.Category;
"""
pd.read_sql(query, conn)

,Month,Category,Target,ActualSales,Difference
0,2018-04,Clothing,12000.0,67649.0,55649.0
1,2018-04,Electronics,9000.0,43637.0,34637.0
2,2018-04,Furniture,10400.0,41541.0,31141.0
3,2018-05,Clothing,12000.0,44338.0,32338.0
4,2018-05,Electronics,9000.0,61765.0,52765.0
5,2018-05,Furniture,10500.0,32664.0,22164.0
6,2018-06,Clothing,12000.0,40292.0,28292.0
7,2018-06,Electronics,9000.0,43234.0,34234.0
8,2018-06,Furniture,10600.0,26489.0,15889.0
9,2018-07,Clothing,14000.0,13060.0,-940.0


Months when the Target is not met:

In [172]:
query = """
SELECT strftime('%Y-%m', s.MonthofOrderDate) AS Month,
       s.Category,
       s.Target,
       SUM(od.Amount * od.Quantity) AS ActualSales,
       SUM(od.Amount * od.Quantity) - s.Target AS Difference
FROM sales_target s
LEFT JOIN orders o
  ON strftime('%Y-%m', o.OrderDate) = strftime('%Y-%m', s.MonthofOrderDate)
LEFT JOIN order_details od
  ON o.OrderID = od.OrderID
 AND od.Category = s.Category
GROUP BY s.MonthofOrderDate, s.Category, s.Target
HAVING Difference < 0
ORDER BY s.MonthofOrderDate, s.Category;
"""
pd.read_sql(query, conn)


,Month,Category,Target,ActualSales,Difference
0,2018-07,Clothing,14000.0,13060.0,-940.0
